# Imports

In [24]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import numpy as np
import pygame
from math import *

In [2]:
n = 2                                            # Number of player(alive)
m = 1                                  # Total number of sources that can deal damage    
l = 0
Dam = [1]  
k=2*n+m+l
const_1= 0.668
A = [1,1,const_1,1,1]

def Event(A):
    Damage = [0]*n
    counts = 10000
    qc = QuantumCircuit()
    q = QuantumRegister(2*(n+l+m)+1+n, 'q')
    c = ClassicalRegister(n, 'c')    
    qc.add_register(q)
    qc.add_register(c)
    for i in range (2*n+m+l+1):
        if i <n:                                                               
            qc.u3(A[i]*np.pi, 0, 0, q[i])
            qc.x(q[i+n])
            qc.u3(A[i+n]*np.pi, 0, 0, q[i+n])
        if n<=i<n+m:                                                            
            qc.x(q[i+n])
            qc.u3(A[i+n]*(1-Dam[i-n])*np.pi, 0, 0, q[i+n])
        if n+m<=i<n+m+l:
            qc.x(q[i+n])
            qc.u3(A[i+n]*np.pi, 0, 0, q[i+n])
        if i == n+m+l:
            qc.mct(q[2*n:2*n+l+m], q[2*n+m+l+1], q[3*n+m+l+2:3*n+2*l+2*m+1])
        if n+m+l<i<2*n+m+l+1:        
            qc.ccx(q[i-(n+m+l+1)], q[i-(m+l+1)], q[2*n+m+l])           
            qc.ccx(q[2*n+m+l], q[2*n+m+l+1], q[i+n+1])
            qc.ccx(q[i-(n+m+l+1)], q[i-(m+l+1)], q[2*n+m+l])
            qc.measure(q[i+1+n], c[2*n+m+l-i])
    qc.draw()
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend,shots = counts)
    job_result = job.result()
    A = list(job_result.get_counts(qc).keys())
    for i in A:
        B = list(i)
        for j in range (n):
            if B[j] == '1':
                Damage[j]+= int(job_result.get_counts(qc).get(i)) /counts
    return Damage

print(Event(A))

[0.2483, 0]


# Intializarion of Data and Constant

In [90]:
xmin = ymin = 0

ymax = 700
p_y = 0.05*ymax
d_x = ymax/2-p_y#450
xmax = d_x*3
r_1 = 0.320*ymax                                        # Constant for the division between the circuit and gates
s_d = 0.112*ymax                                           # Constatnt for grids in the first 2 box 
Gate = 0
x_temp= y_temp = 0

x_c = []
y_c1 = []
y_c2 = []
x_gate = []
y_gate_1 = []
y_gate_2 = []
for i in range (4):
    y_c1.append(0.09*ymax+0.08*ymax*i)
    x_c.append((0.965+0.1*i)*ymax)
    y_c2.append(0.59*ymax+0.08*ymax*i)
for i in range(3):
    x_gate.append((0.965+0.1*i)*ymax)
    y_gate_1.append(ymax*0.435)
    y_gate_2.append(ymax*0.935)
y_c = [y_c1,y_c2]
y_gate=[y_gate_1,y_gate_2]
x_cancle = 1.265*ymax
y_cancle = [ymax*0.435,ymax*0.935]
H = pygame.image.load(r'Resource\H.jpg')
X = pygame.image.load(r'Resource\X.jpg')
D = pygame.image.load(r'Resource\D.jpg')
C = pygame.image.load(r'Resource\C.png')
coin = pygame.image.load(r'Resource\coin.jpg')
player1_image = pygame.image.load(r'Resource\player.png')
player2_image = pygame.image.load(r'Resource\player.jpg')
one_image = pygame.image.load(r'Resource\1.jpg')
two_image = pygame.image.load(r'Resource\2.jpg')
three_image = pygame.image.load(r'Resource\3.jpg')
blur_full = pygame.image.load(r'Resource\Blur_Full.jpg')
blur_half = pygame.image.load(r'Resource\Blur_Half.jpg')
bomb = pygame.image.load(r'Resource\bomb.jpg')
blast = pygame.image.load(r'Resource\blast.jpg')





edge = int(0.07*ymax)
H = pygame.transform.scale(H, (edge, edge))
X = pygame.transform.scale(X, (edge, edge))
D = pygame.transform.scale(D, (edge, edge))
C = pygame.transform.scale(C, (edge, edge))
coin = pygame.transform.scale(coin, (floor(edge/2), floor(edge/2)))
player1_image = pygame.transform.scale(player1_image, (edge, edge))
player2_image = pygame.transform.scale(player2_image, (edge, edge))
one_image = pygame.transform.scale(one_image, (2*edge, 2*edge))
two_image = pygame.transform.scale(two_image, (2*edge, 2*edge))
three_image = pygame.transform.scale(three_image, (2*edge, 2*edge))
blur_full = pygame.transform.scale(blur_full, (int(xmax), int(ymax)))
blur_half = pygame.transform.scale(blur_half, (int(xmax), int(ymax/2)))
bomb = pygame.transform.scale(bomb, (edge, edge))
blast = pygame.transform.scale(blast, (edge, edge))


countdown_image = [three_image,two_image,one_image]
player_image = [player1_image,player2_image]
Gates_image = [H,X,D]
def grid_pos(x,y):
        temp_x_pos = []
        temp_y_pos = []
        for i in range(0,4):
            temp_x_pos.append(s_d*i+x)
            temp_y_pos.append(s_d*i+y)
        temp_x_pos.sort()
        temp_y_pos.sort()
        return(temp_x_pos,temp_y_pos)
coord_grid= [[[],[]],[[],[]]]
for i in range(2):
        for j in range(2):
            coord_grid[i][j] = grid_pos(d_x*i,j*ymax/2+p_y)

# Structure of the Board

In [4]:
def Structure():
    
    pygame.draw.line(win, (0,0,0), (0, ymax/2),(xmax,ymax/2), 2)   
    pygame.draw.line(win, (0,0,0), (d_x,p_y),(d_x,ymax/2), 5)
    pygame.draw.line(win, (0,0,0), (d_x*2,p_y),(d_x*2,ymax/2), 5)  
    pygame.draw.line(win, (0,0,0), (d_x,ymax/2+p_y),(d_x,ymax), 5)  
    pygame.draw.line(win, (0,0,0), (d_x*2,ymax/2+p_y),(d_x*2,ymax), 5)  

    pygame.draw.line(win, (0,0,0), (0,p_y),(xmax,p_y), 2)
    pygame.draw.line(win, (0,0,0), (0,ymax/2+p_y),(xmax,ymax/2+p_y), 2)
    # Division between the last box
    pygame.draw.line(win, (0,0,0), (d_x*2,r_1+p_y),(xmax,r_1+p_y), 2)
    pygame.draw.line(win, (0,0,0), (d_x*2,r_1+p_y+ymax/2),(xmax,r_1+p_y+ymax/2), 2)
    
    
    # Section in First and second Box 
    def grid(x,y):
        for i in range(1,4):
            pygame.draw.line(win, (0,0,0), (x,s_d*i+y),(x+d_x,s_d*i+y), 2)
            pygame.draw.line(win, (0,0,0), (s_d*i+x,y),(s_d*i+x,y+d_x), 2)
        return()
    
    for i in range(2):
        for j in range(2):
            grid(d_x*i,j*ymax/2+p_y)
            
            
    
    #Gates Position
    def gates(x,y):
        win.blit(H, (x+0.03*ymax, y+0.03*ymax)) 
        win.blit(X, (x+0.13*ymax, y+0.03*ymax)) 
        win.blit(D, (x+0.23*ymax, y+0.03*ymax))
        win.blit(C, (x+0.33*ymax, y+0.03*ymax))
                          
        return()
    gates(0.9*ymax,ymax*0.370)
    gates(0.9*ymax,ymax*0.870)
    
    # Qubit Wire
    for i in range (4):
        pygame.draw.line(win, (0,0,0), (d_x*2+0.0148148*xmax,0.09*ymax+0.08*ymax*i),((1-0.0148148)*xmax,0.09*ymax+0.08*ymax*i), 2)
        pygame.draw.line(win, (0,0,0), (d_x*2+0.0148148*xmax,0.59*ymax+0.08*ymax*i),((1-0.0148148)*xmax,0.59*ymax+0.08*ymax*i), 2)
    
    for i in range(len(x_c)):
        for j in range(len(y_c1)):
            pygame.draw.circle(win, (0,0,0), (x_c[i],y_c1[j]), 3)
            pygame.draw.circle(win, (0,0,0), (x_c[i],y_c2[j]), 3)
    
    return()

# Important Functions

In [5]:
def Gate_Intraction_part1(player_active):
    #gate = 0
    global Gate
    global temp_cons1
    global temp_cons2
    global player_mat
    global check_1
    xm_pos,ym_pos=pygame.mouse.get_pos()
    

    if pygame.mouse.get_pressed()[0]:
        
        if abs(xm_pos-x_cancle)<=(0.035*ymax) and abs(ym_pos - y_cancle[int(player_active-1)])<=(0.035*ymax):
            player_mat[player_active-1] = np.zeros((4,4))
            check_1 = True
        
        if Gate==0:
            for i in range(3):
                if abs(xm_pos-x_gate[i])<=(0.035*ymax) and abs(ym_pos - y_gate[int(player_active-1)][i])<=(0.035*ymax):
                    Gate = i+1
                    
        elif Gate ==3:
            for i in range(4):
                    for j in range(4):
                        if abs(xm_pos-x_c[i])<=(0.035*ymax) and abs(ym_pos - y_c[player_active-1][j])<=(0.035*ymax):
                            xm_pos = x_c[i]
                            ym_pos =  y_c[player_active-1][j]
                            temp_cons1 = j
                            temp_cons2 = i
                            for k in range(4):
                                if player_mat[player_active-1][k,i] !=0:
                                    #print(k,i)
                                    pygame.draw.line(win, (100,100,100), (x_c[i],y_c[player_active-1][j]),(x_c[i],y_c[player_active-1][k]), 3)
            win.blit(Gates_image[Gate-1], (xm_pos-0.035*ymax, ym_pos-0.035*ymax))
        else:
            for i in range(4):
                    for j in range(4):
                        if abs(xm_pos-x_c[i])<=(0.035*ymax) and abs(ym_pos - y_c[player_active-1][j])<=(0.035*ymax):
                            xm_pos = x_c[i]
                            ym_pos =  y_c[player_active-1][j]
                            temp_cons1 = j
                            temp_cons2 = i
            win.blit(Gates_image[Gate-1], (xm_pos-0.035*ymax, ym_pos-0.035*ymax))
    else:
        if Gate ==3 and abs(xm_pos-x_c[temp_cons2])<=(0.035*ymax) and abs(ym_pos - y_c[player_active-1][temp_cons1])<=(0.035*ymax):
            temp_const_0 = len(np.where(np.array(player_mat[player_active-1][:,temp_cons2]).flatten()!=0)[0])
            temp_const_1 = len(np.where(np.array(player_mat[player_active-1][:,temp_cons2]).flatten()==1)[0])
            temp_const_2 = len(np.where(np.array(player_mat[player_active-1][:,temp_cons2]).flatten()==2)[0])
            temp_const_3 = len(np.where(np.array(player_mat[player_active-1][:,temp_cons2]).flatten()==3)[0])
            if  temp_const_0 != 0 and temp_const_1+temp_const_2 ==1:
                if temp_const_1 ==1 and temp_const_3<1:
                    player_mat[player_active-1][temp_cons1,temp_cons2] = Gate
                    check_1 = True
                elif temp_const_2 ==1:
                    player_mat[player_active-1][temp_cons1,temp_cons2] = Gate
                    check_1 = True
            
        elif Gate!= 0 and abs(xm_pos-x_c[temp_cons2])<=(0.035*ymax) and abs(ym_pos - y_c[player_active-1][temp_cons1])<=(0.035*ymax) :
            if len(np.where(np.array(player_mat[player_active-1][:,temp_cons2]).flatten()==3)[0]) ==0:
                player_mat[player_active-1][temp_cons1,temp_cons2] = Gate
                check_1 = True
        Gate = 0 
    for i in range (len(x_c)):
        for j in range (len(x_c)):
            if player_mat[player_active-1][i,j]==3:
                for k in range (4):
                    if player_mat[player_active-1][k,j]!=0 and player_mat[player_active-1][k,j]!=3:
                        #print(i,k)
                        pygame.draw.line(win, (100,100,100), (x_c[j],y_c[player_active-1][i]),(x_c[j],y_c[player_active-1][k]), 3)
                win.blit(Gates_image[int(player_mat[player_active-1][i,j])-1], (x_c[j]-0.035*ymax, y_c[player_active-1][i]-0.035*ymax))
    
    
    for i in range (len(x_c)):
        for j in range (len(x_c)):
                  
            if player_mat[player_active-1][i,j]!=0and player_mat[player_active-1][i,j]!=3:
                win.blit(Gates_image[int(player_mat[player_active-1][i,j])-1], (x_c[j]-0.035*ymax, y_c[player_active-1][i]-0.035*ymax))
        
    return()

In [6]:
def Gate_Intraction_part2(player_active):
    #gate = 0
    global Gate
    global temp_cons1
    global temp_cons2
    global player_mat
    global check_1
    xm_pos,ym_pos=pygame.mouse.get_pos()
    

    if pygame.mouse.get_pressed()[0]:
        
        if abs(xm_pos-x_cancle)<=(0.035*ymax) and abs(ym_pos - y_cancle[int(player_active-1)])<=(0.035*ymax):
            player_mat[player_active-1] = np.zeros((4,4))
            check_1 = True
        
        if Gate==0:
            for i in range(3):
                if abs(xm_pos-x_gate[i])<=(0.035*ymax) and abs(ym_pos - y_gate[int(player_active-1)][i])<=(0.035*ymax):
                    Gate = i+1
        if Gate == 2:
            for i in range(4):
                    for j in range(4):
                        if abs(xm_pos-x_c[i])<=(0.035*ymax) and abs(ym_pos - y_c[player_active-1][j])<=(0.035*ymax):
                            xm_pos = x_c[i]
                            ym_pos =  y_c[player_active-1][j]
                            temp_cons1 = j
                            temp_cons2 = i
            win.blit(Gates_image[Gate-1], (xm_pos-0.035*ymax, ym_pos-0.035*ymax))
    else:            
        if Gate == 2 and abs(xm_pos-x_c[temp_cons2])<=(0.035*ymax) and abs(ym_pos - y_c[player_active-1][temp_cons1])<=(0.035*ymax) :
            if len(np.where(np.array(player_mat[player_active-1][:,temp_cons2]).flatten()==3)[0]) ==0:
                player_mat[player_active-1][temp_cons1,temp_cons2] = Gate
                check_1 = True
        Gate = 0 
    for i in range (len(x_c)):
        for j in range (len(x_c)):
            if player_mat[player_active-1][i,j]==3:
                for k in range (4):
                    if player_mat[player_active-1][k,j]!=0 and player_mat[player_active-1][k,j]!=3:
                        #print(i,k)
                        pygame.draw.line(win, (100,100,100), (x_c[j],y_c[player_active-1][i]),(x_c[j],y_c[player_active-1][k]), 3)
                win.blit(Gates_image[int(player_mat[player_active-1][i,j])-1], (x_c[j]-0.035*ymax, y_c[player_active-1][i]-0.035*ymax))
    
    
    for i in range (len(x_c)):
        for j in range (len(x_c)):
                  
            if player_mat[player_active-1][i,j]!=0and player_mat[player_active-1][i,j]!=3:
                win.blit(Gates_image[int(player_mat[player_active-1][i,j])-1], (x_c[j]-0.035*ymax, y_c[player_active-1][i]-0.035*ymax))
        
    return()

In [7]:
def Coin_Position(player_active):
    global check_1
    if check_1:
        bool_grid[player_active-1] = val(result(circuit(player_mat[player_active-1]),100))
        check_1 = False
    coord_use = coord_grid[player_active-1][player_active-1]   
    
    for i in range (4):
        for j in range(4):
            if bool_grid[player_active-1][i,j]==1:
                win.blit(coin, (coord_use[0][j]+0.075*ymax, coord_use[1][i]+0.075*ymax)) 
    return()

In [8]:
def circuit(mat):
    qc = QuantumCircuit(4,4)
    for i in range(4):
        temp_const_1 = list(np.where(np.array(mat[:,i]).flatten()==1)[0])
        temp_const_2 = list(np.where(np.array(mat[:,i]).flatten()==2)[0])
        temp_const_3 = list(np.where(np.array(mat[:,i]).flatten()==3)[0])
        
        
        if len(temp_const_1) != 0:
            if len(temp_const_3) ==0:
                qc.h(temp_const_1)
            else:
                qc.ch(temp_const_3,temp_const_1)
        if len(temp_const_2) != 0:
            qc.mcx(temp_const_3,temp_const_2)
        
    qc.measure([0,1,2,3],[0,1,2,3])
    return(qc)

In [9]:
def result(circuit,counts):
    qc = circuit
    
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend,shots = counts)
    job_result = job.result()
    result = job_result.get_counts(qc)
    return(result)

In [10]:
def val(result):
    p = np.array([0]*16)
    for i in range(16):
        x = np.binary_repr(i, width=4)
        if x in result:
            p[i] = 1
    p = p.reshape(4, 4)
    return(p)

In [11]:
def position_check(bool_grid,pos):
    check_2 = False
    for i in range(-1,2):
        for j in range(-1,2):
            if -1<pos[0]-i<4 and -1<pos[1]-j<4:
                if bool_grid[pos[0]-i,pos[1]-j] == 1 and i+j+i*j!=0:
                    check_2 = True
                    break
    return((bool_grid[pos[1],pos[0]] ==1 and len(np.where(bool_grid==1)[0])!=1) or check_2 and len(np.where(bool_grid==1)[0])==1)
    
    

In [12]:
def Coin_pos_Check(player_active):
    global check_3
    keys=pygame.key.get_pressed()
    if keys[pygame.K_RETURN]!= True and check_3 == 1:
        check_3 = 2

    if keys[pygame.K_RETURN]and check_3==0 and position_check(bool_grid[player_active-1],player_pos[player_active-1]):
        check_3 = 1
    
    elif keys[pygame.K_RETURN] and check_3 == 2:
        check_3 = 3
    

In [13]:
def player_show_part1(player_active,player_passive):
    coord_use_active = coord_grid[player_active-1][player_active-1]
    win.blit(player_image[player_active-1], (coord_use_active[0][player_pos[player_active-1][0]]+0.025*ymax, coord_use_active[1][player_pos[player_active-1][1]]+0.025*ymax))
    coord_use_passive = coord_grid[player_passive-1][player_active-1]
    win.blit(player_image[player_passive-1], (coord_use_passive[0][player_pos[player_passive-1][0]]+0.025*ymax, coord_use_passive[1][player_pos[player_passive-1][1]]+0.025*ymax))

In [14]:
def player_show_part2(player_active,player_passive):
    coord_use_active = coord_grid[player_active-1][player_passive-1]
    win.blit(player_image[player_active-1], (coord_use_active[0][player_pos[player_active-1][0]]+0.025*ymax, coord_use_active[1][player_pos[player_active-1][1]]+0.025*ymax))
    coord_use_passive = coord_grid[player_passive-1][player_passive-1]
    win.blit(player_image[player_passive-1], (coord_use_passive[0][player_pos[player_passive-1][0]]+0.025*ymax, coord_use_passive[1][player_pos[player_passive-1][1]]+0.025*ymax))

In [15]:
def Bomb_Position(player_active,player_passive):
    global check_1
    if check_1:
        bool_grid[player_passive-1] = val(result(circuit(player_mat[player_passive-1]),1))
        check_1 = False
    coord_use = coord_grid[player_active-1][player_passive-1]   
    
    for i in range (4):
        for j in range(4):
            if bool_grid[player_passive-1][i,j]==1:
                win.blit(bomb, (coord_use[0][j]+0.025*ymax, coord_use[1][i]+0.025*ymax)) 
    return()

In [101]:
'''
def Blast_Position(player_active,player_passive):
    coord_use_1 = coord_grid[player_active-1][player_passive-1] 
    coord_use_2 = coord_grid[player_active-1][player_active-1]   
    global blast_cord
    blast_cord = []
    for i in range (4):
        for j in range(4):
            if bool_grid[player_passive-1][i,j]==1:
                temp_blast_1= i
                temp_blast_2= j
                blast_cord = [j,i]
                win.blit(blast, (coord_use_1[0][j]+0.025*ymax, coord_use_1[1][i]+0.025*ymax))
                win.blit(blast, (coord_use_2[0][j]+0.025*ymax, coord_use_2[1][i]+0.025*ymax))
    if temp_blast_2-1>=0:
                win.blit(blast, (coord_use_1[0][temp_blast_2-1]+0.025*ymax, coord_use_1[1][temp_blast_1]+0.025*ymax))
                win.blit(blast, (coord_use_2[0][temp_blast_2-1]+0.025*ymax, coord_use_2[1][temp_blast_1]+0.025*ymax))
    if temp_blast_1-1>=0:
                win.blit(blast, (coord_use_1[0][temp_blast_2]+0.025*ymax, coord_use_1[1][temp_blast_1-1]+0.025*ymax))
                win.blit(blast, (coord_use_2[0][temp_blast_2]+0.025*ymax, coord_use_2[1][temp_blast_1-1]+0.025*ymax))
    if temp_blast_2+1<4:
                win.blit(blast, (coord_use_1[0][temp_blast_2+1]+0.025*ymax, coord_use_1[1][temp_blast_1]+0.025*ymax))
                win.blit(blast, (coord_use_2[0][temp_blast_2+1]+0.025*ymax, coord_use_2[1][temp_blast_1]+0.025*ymax))
    if temp_blast_1+1<4:
                win.blit(blast, (coord_use_1[0][temp_blast_2]+0.025*ymax, coord_use_1[1][temp_blast_1+1]+0.025*ymax))
                win.blit(blast, (coord_use_2[0][temp_blast_2]+0.025*ymax, coord_use_2[1][temp_blast_1+1]+0.025*ymax))
    return()
'''

'\ndef Blast_Position(player_active,player_passive):\n    coord_use_1 = coord_grid[player_active-1][player_passive-1] \n    coord_use_2 = coord_grid[player_active-1][player_active-1]   \n    global blast_cord\n    blast_cord = []\n    for i in range (4):\n        for j in range(4):\n            if bool_grid[player_passive-1][i,j]==1:\n                temp_blast_1= i\n                temp_blast_2= j\n                blast_cord = [j,i]\n                win.blit(blast, (coord_use_1[0][j]+0.025*ymax, coord_use_1[1][i]+0.025*ymax))\n                win.blit(blast, (coord_use_2[0][j]+0.025*ymax, coord_use_2[1][i]+0.025*ymax))\n    if temp_blast_2-1>=0:\n                win.blit(blast, (coord_use_1[0][temp_blast_2-1]+0.025*ymax, coord_use_1[1][temp_blast_1]+0.025*ymax))\n                win.blit(blast, (coord_use_2[0][temp_blast_2-1]+0.025*ymax, coord_use_2[1][temp_blast_1]+0.025*ymax))\n    if temp_blast_1-1>=0:\n                win.blit(blast, (coord_use_1[0][temp_blast_2]+0.025*ymax, coord

In [102]:
def Blast_Position(blast_end,player_active,player_passive):
    coord_use_1 = coord_grid[player_active-1][player_passive-1] 
    coord_use_2 = coord_grid[player_active-1][player_active-1] 
    for i in blast_end:
        win.blit(blast, (coord_use_1[0][i[0]]+0.025*ymax, coord_use_1[1][i[1]]+0.025*ymax))
        win.blit(blast, (coord_use_2[0][i[0]]+0.025*ymax, coord_use_2[1][i[1]]+0.025*ymax))
    return()

In [103]:
def blast_pos_list(player_passive):
    t = []
      
    for i in range (4):
        for j in range(4):
            if bool_grid[player_passive-1][i,j]==1:
                blast_end = [j,i]
    t.append(blast_end)
    if blast_end[0]-1>=0:
                t.append([blast_end[0]-1,blast_end[1]])
    if blast_end[1]-1>=0:
                t.append([blast_end[0],blast_end[1]-1])
    if blast_end[0]+1<4:
                t.append([blast_end[0]+1,blast_end[1]])
    if blast_end[1]+1<4:
                t.append([blast_end[0],blast_end[1]+1])
    return(t)
    

In [104]:
def Bomb_Position_end(player_active,player_passive):
    coord_use_1 = coord_grid[player_active-1][player_passive-1] 
    coord_use_2 = coord_grid[player_active-1][player_active-1]   

    
    for i in range (4):
        for j in range(4):
            if bool_grid[player_passive-1][i,j]==1:
                win.blit(bomb, (coord_use_1[0][j]+0.025*ymax, coord_use_1[1][i]+0.025*ymax))
                win.blit(bomb, (coord_use_2[0][j]+0.025*ymax, coord_use_2[1][i]+0.025*ymax))
    return()

In [105]:
def bomb_exit():
    global check_3
    keys=pygame.key.get_pressed()
    if keys[pygame.K_RETURN]!= True and check_3 == 5:
        check_3 = 6

    if keys[pygame.K_RETURN]and check_3==4 :
        check_3 = 5
    
    elif keys[pygame.K_RETURN] and check_3 == 6:
        rune = False
        check_3 = 7

In [106]:
def Player_updated_pos(player_active):
    temp_const_3 = int(list(result(circuit(player_mat[player_active-1]),1).keys())[0],2)
    player_pos[player_active-1][1] = temp_const_3//4
    player_pos[player_active-1][0] = temp_const_3%4
    return()

    

()

# Pygame

In [123]:
player_mat= [np.zeros((4,4)),np.zeros((4,4))]
player_active = 1
bool_grid = np.array([0]*16)
bool_grid = bool_grid.reshape(4, 4)
bool_grid = [bool_grid,bool_grid]
temp_mat = 3*np.ones((4,4))
check_1 = True
check_3 = 0
check_4 = 0
Turn = 2
player1_pos = [0,0]
player2_pos = [0,0]
player_pos=[player1_pos , player2_pos]
pygame.init()
win = pygame.display.set_mode((int(xmax),int(ymax)))
pygame.display.set_caption("QGame_1")
time_delay = 10
run = True

while run:
    ##############
    pygame.time.delay(time_delay)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    
    win.fill((255,255,255))
    Structure()
    ##############
    if check_3 <3:
        if Turn ==1:
            player_defence = 1
            player_attack = 2
        else:
            player_defence = 2
            player_attack = 1
            

        Gate_Intraction_part1(player_defence)
        player_show_part1(player_defence,player_attack)
        Coin_Position(player_defence)
        #Player_updated_pos(player_active)
        Coin_pos_Check(player_defence)
        
    if check_3 ==3:
        win.blit(blur_full, (0, 0))
        if check_4 <100:
            win.blit(countdown_image[0], (xmax/2-edge, ymax/2-edge))
        if 100<=check_4 <200:
            win.blit(countdown_image[1], (xmax/2-edge, ymax/2-edge))
        if 200<=check_4 <300:
            win.blit(countdown_image[2], (xmax/2-edge, ymax/2-edge))
        if check_4 == 300:
            check_3 = 4
        check_4 +=1
        check_1 = True
       
    if 4<=check_3 <7:
        win.blit(blur_half, (0, (player_defence-1)*ymax/2+5))
        player_show_part2(player_defence,player_attack)
        Gate_Intraction_part2(player_attack)
        Bomb_Position(player_defence,player_attack)
        bomb_exit()
        check_4 = 0
    if check_3 ==7:
        win.blit(blur_full, (0, 0))
        if check_4 <100:
            win.blit(countdown_image[0], (xmax/2-edge, ymax/2-edge))
        if 100<=check_4 <200:
            win.blit(countdown_image[1], (xmax/2-edge, ymax/2-edge))
        if 200<=check_4 <300:
            win.blit(countdown_image[2], (xmax/2-edge, ymax/2-edge))
        if check_4 == 300:
            check_3 = 8
        check_4 +=1
    if check_3==8:
        Player_updated_pos(player_defence)
        check_3 += 1
        check_4 = 0
    if check_3==9:
        player_show_part1(player_defence,player_attack)
        player_show_part2(player_defence,player_attack)
        Bomb_Position_end(player_defence,player_attack)
        check_4+=1
        if check_4 ==50:
            check_3 += 1
    if check_3 == 10:
        blast_cord = blast_pos_list(player_attack)
        check_3+=1
        check_4 = 0
    if check_3 == 11:
        player_show_part1(player_defence,player_attack)
        player_show_part2(player_defence,player_attack)
        #Blast_Position(player_defence,player_attack)
        Blast_Position(blast_cord,player_defence,player_attack)
        check_4+=1
        print(check_4)
        if check_4 ==10:
            finish_term(player_defence,blast_cord)
            
            

    pygame.display.update()
pygame.quit()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [98]:
blast_cord

[<Surface(49x49x24 SW)>, [1, 2], [2, 1], [3, 2], [2, 3]]

In [13]:
def Player_updated_pos(player_active):
    global check_3
    keys=pygame.key.get_pressed()
    if keys[pygame.K_RETURN]!= True and check_3 == 1:
        check_3 = 2

    if keys[pygame.K_RETURN]and check_3==0 and position_check(bool_grid[player_active-1],player_pos[player_active-1]):
        temp_const_3 = int(list(result(circuit(player_mat[player_active-1]),1).keys())[0],2)
        player_pos[player_active-1][1] = temp_const_3//4
        player_pos[player_active-1][0] = temp_const_3%4
        check_3 = 1
    elif keys[pygame.K_RETURN] and check_3 == 2:
        check_3 = 0
    

In [113]:
def finish_term(player_active,blast_cord):
    if player_pos[player_active-1] in blast_cord:
        run = False
    else:
        turm = (turm+1)%2+1
    return()

In [112]:
2%2

0

In [122]:
player_pos[player_defence-1]in blast_cord

True

In [117]:
blast_cord

[[0, 1], [0, 0], [1, 1], [0, 2]]